In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
import numpy as np
import keras
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from shapely.geometry import Polygon, box

import rasterio
from rasterio import plot as rastPlt
from rasterio.mask import mask as rasterMask
from rasterio.plot import reshape_as_raster

# Data preparation
### Fetch data from file

In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS.tif"
pathValidation = folderName + "Population_GHSL.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)

lightMask = dataRadiance>0
lightCoords = dataCoords[lightMask]

print('Tiles:',dataCoords.shape[0])
print('Light Tile:',lightCoords.shape[0])


#### Fetch images

In [ ]:
with rasterio.open(pathSatellite) as f:
  trainData, _ = utils.coordsToImgsFormated(f, lightCoords, res=64)
print(trainData.shape)

#### Prepare Validation

In [ ]:
with rasterio.open(pathValidation) as f:
  testTile, _ = rasterMask(f, [Polygon(dataCoords[1000])], crop=True)
print('Validation shape:',testTile.shape) # shape sample

print('Process validation...')
getValid = lambda data : [ int(250/255 < img.max()) for img in data ]
resultValid = utils.scanSatellite(pathValidation, dataCoords, getValid, batch=1000, res=testTile.shape[1])

## Autoencoder + SVM

In [ ]:
autoencoder = keras.models.load_model('model/autoencoder_64px_encoder_1024')
encoder = keras.Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('encoder').output)

trainDataFormated = encoder.predict(trainData)
validation = dataRadiance[lightMask]

In [ ]:
svm = OneClassSVM(gamma='scale', kernel='rbf', nu=0.8, tol=1e-3)
svm = svm.fit(trainDataFormated, sample_weight=dataRadiance[lightMask])

# Analyse model

In [ ]:
print('Process score...')
getScore = lambda data : [1 if i == 1 else 0 for i in svm.predict(encoder.predict(data, verbose=0))]
result = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=100, res=64)

# Confusion Matrix

In [ ]:
print('Process confustion matrix...')
print('total',len(result))
confusionMatrix = confusion_matrix(resultValid, result)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

# Display the analysed results

In [ ]:
resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, result, resultValid)

fig, axs = matPlt.subplots(1,3, dpi=240)
with rasterio.open(pathSatellite) as s: utils.displayTiles([s.read()], [s.transform],axs[0])

axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())

with rasterio.open(pathValidation) as p: rastPlt.show(p, ax=axs[2])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())

utils.displayTiles([resultImg], [resultMeta], axs[1])

## Test

In [ ]:
img = np.copy(resultImg)
img = img[0:3,:,:]
img = img.transpose([1, 2, 0])
with rasterio.open(pathNight) as f:
  profile = f.profile
img = reshape_as_raster(img)
profile.update(count=3)
print(profile)
print(img.shape)
with rasterio.open(folderName + 'autoencoder_svm_result.tif', 'w', **profile) as f:
  f.write(img)